# Лекция 10 "Линзы, Призмы и Template Haskell"

__План:__
- Lens
  - Что это за зверь ?
  - Законы для линз
  - Пример работы
  - Traversal
  - Prisms
- Макросы
  - Плюсы и минусы
- Template Haskell
  - Lens generator
  - Генерация функций
  - Главное правило для макросов
  - -XQuasiQuotes

## Lens

### Что это? Зачем?

В _Haskell_ не всё так просто, когда дело заходит о полях структур. Если получить значение можно, то вот изменить его не самая тривиальная задача. А представьте, что внутри одной структуры лежит другая, в которой тоже что-то лежит, и вы приходите к нервному расстройству. 

Пример:

```
data Person = Person 
    { name    :: String
    , age     :: Int
    , address :: Address
    }
data Address = Address 
    { house  :: Int
    , street :: String
    , city   :: String
    }
james = Person {
    name    = "James",
    age     = 28,
    address = Address {
        house  = 42,
        street = "Some Road",
        city   = "London"
    }
}

houseNumber =
  house (address james)

updatedJames =
  james {
    address =
      (address james) { house = 29 }
    }

setHouse person value =
  person { address = (address person) { house = value }  }
setStreet person value =
  person { address = (address person) { street = value }  }

newPerson1 = setHouse person 45
newPerson2 = setStreet person "New Street"
```

Поэтому заботливые создатели ввели такое понятие как __Линзы__. Их используют, чтобы получать доступ к полям, даже тем, которые лежат на самом нижнем уровне. 

Вот как выглядит общий тип __Lens__ в _Haskell_:

```
type Lens  s t a b = forall f. Functor f => (a -> f b) -> (s -> f t)
type Lens' s   a   = Lens s s a a
type Lens' s   a   = forall f. Functor f => (a -> f a) -> (s -> f s)

newtype Const a x = Const { getConst :: a }
instance Functor (Const a) where
    fmap _ (Const v) = Const v
-- view :: ((a -> Const a a) -> (s -> Const a s)) -> s -> a
view :: Lens' s a -> s -> a
view l s = getConst $ l Const s

newtype Identity a = Identity { runIdentity :: a }
instance Functor Identity where
    fmap f (Identity x) = Identity (f x)
-- over :: ((a -> Identity a) -> s -> Identity s) -> (a -> a) -> s -> s
over :: Lens' s a -> (a -> a) -> s -> s
over l fn s = runIdentity $ l (Identity . fn) s

-- set :: ((a -> Identity a) -> s -> Identity s) -> a -> s -> s
set :: Lens' s a -> a -> s -> s
set l a s = runIdentity $ l (Identity . const a) s

-- lens :: (s -> a) -> (s -> a -> s) -> (a -> f a) -> s -> f s
lens :: (s -> a) -> (s -> a -> s) -> Lens' s a
lens get set f s = set s <$> f (get s)
```

* __NOTE:__ __lens__ library is really heavy. It has a lot of dependencies. If you're writing your own library you might want to use lighter alternatives which are, happily, completely compatible with __lens__ library!

  - [Hackage: microlens — types, simple operators](https://hackage.haskell.org/package/microlens)
  - [Hackage: microlens-ghc — microlens + containers, arrays, etc.](https://hackage.haskell.org/package/microlens-ghc)
  - [Hackage: microlens-mtl — operators for mtl monads](https://hackage.haskell.org/package/microlens-mtl)
  - [Hackage: microlens-th — TemplateHaskell macros](https://hackage.haskell.org/package/microlens-th)
  - [Hackage: microlens-platform — All above](https://hackage.haskell.org/package/microlens-platform)


### __Lens__ Laws

1. You get bach what you put in:  
`view l (set l field obj)   ≡ field`

2. Putting back what you got doesn't change anything:  
`set l (view l obj) obj   ≡ obj`

3. Setting twice is the same as setting once:  
`set l field (set l field obj) ≡ set l field obj`

### Работа с линзами

Для работы с линзами ввели библиотеку __`Control.Lens`__. В котором уже реализовано огромное число операций в виде различных операторов (многосимвольных).  
Для этих операторов в свою очередь существует некая закономерность в их обозначении:

1. Operators that begin with __`^`__ are kinds of views. The only example we've seen so far is (__`^.`__) which is view exactly.
2. Operators that end with __`~`__ are like over or set. In fact,
(__`.~`__) == set and (__`%~`__) is over.
3. Operators that have __`.`__ in them are usually somehow «basic»
4. Operators that have __`%`__ in them usually take functions.
5. Operators that have __`=`__ in them are just like their cousins where __`=`__ is replaced by __`~`__, but instead of taking the whole object as an argument, they apply their modifications in a State monad.

```
(.~) :: Lens' s a -> a        -> (s -> s)
(.=) :: Lens' s a -> a        -> State s ()

(%~) :: Lens' s a -> (a -> a) -> (s -> s)
(%=) :: Lens' s a -> (a -> a) -> State s ()
```

Рассмотрим простой пример (игру):

```
initialState :: Game
initialState = Game
    { _score = 0
    , _units =
        [ Unit { _health = 10
               , _position = Point { _x = 3.5
                                   , _y = 7.0 }
               }
        , Unit { _health = 15
               , _position = Point { _x = 1.0
                                   , _y = 1.0 }
               }
        , Unit { _health = 8
               , _position = Point { _x = 0.0
                                   , _y = 2.1 }
               }
        ]
    , _boss = Unit
        { _health = 100
        , _position = Point { _x = 0.0
                            , _y = 0.0 }
        }
    }

score :: Lens' Game Int
score = lens _score (\game v -> game { _score = v })

units :: Lens' Game [Unit]
units = lens _units (\game v -> game { _units = v })

boss :: Lens' Game Unit
boss = lens _boss (\game v -> game { _boss = v })

health :: Lens' Unit Int
health = lens _health (\unit v -> unit { _health = v })

position :: Lens' Unit Point
position = lens _position (\unit v -> unit { _position = v })

x :: Lens' Point Double
x = lens _x (\point v -> point { _x = v })

y :: Lens' Point Double
y = lens _y (\point v -> point { _y = v })

lens :: (s -> a) -> (s -> a -> s) -> Lens' s a
```

Попробуем обратиться к полю Здоровье в структуре Босса:

```
boss . health :: ??? -- Что будет с типом ???
```

Посмотрим на типы оператора __`.`__:

```
(.) :: (b -> c) -> (a -> b) -> (a -> c)
(g . f) x = g (f x)

a ~ (Int  -> f Int)
b ~ (Unit -> f Unit)
c ~ (Game -> f Game)

(.) :: ((Unit -> f Unit) -> (Game -> f Game))
    -> ((Int  -> f Int ) -> (Unit -> f Unit))
    -> ((Int  -> f Int ) -> (Game -> f Game))

-- Более читабельно с помощью линз --
(.) :: Lens' Game Unit 
    -> Lens' Unit Int
    -> Lens' Game Int
```

И вот мы поняли, какой тип будет иметь наша композиция:

```
boss . health :: Lens' Game Int
```

Ну и вообще пару манипуляций с нашей игрой:

```
-- Getters --
ghci> initialState^.score
0
ghci> initialState^.boss.position.x
0.0

-- Setters --
ghci> let thisBoss = initialState^.boss
ghci> thisBoss
Unit {_health = 100, _position = Point {_x = 0.0, _y = 0.0}}
ghci> thisBoss & position.x .~ 1.0
Unit {_health = 100, _position = Point {_x = 1.0, _y = 0.0}}

-- Комбинация с монадами --
import Control.Monad.Trans.Class
import Control.Monad.Trans.State

strike :: StateT Game IO ()
strike = do
    liftIO $ putStrLn "*shink*"
    boss.health -= 10

ghci> newState <- execStateT strike initialState 
*shink*
ghci> newState^.boss.health
90
```

Теперь обращение к полям выглядит так же, как и в других ЯП.


### Traversal

Линзы помогают фокусироваться на одном поле. А что если у нас в поле лежит массив ?? На помощь приходит __`Traversal`__:

```
type Traversal s t a b 
    = forall f . Applicative f => (a -> f b) -> (s -> f t)
type Traversal' obj field 
    = forall f . Applicative f => (field -> f field) -> (obj -> f obj)

class (Functor t, Foldable t) => Traversable t where
  traverse :: Applicative f => (a -> f b) -> t a -> f (t b)

traverse :: (Applicative f, Traversable t) => (a -> f b) -> t a -> f (t b)
traverse :: (Applicative f, Traversable t) => (a -> f b) -> (t a -> f (t b))
traverse :: Traversal (t a) (t b) a b 

traversed :: Traversable t => Traversal' (t a) a
traversed = traverse
```

Теперь мы можем работать сразу со всеми значениями в поле-массиве:

```
fireBreath :: StateT Game IO ()
fireBreath = do
    lift $ putStrLn "*rawr*"
    units.traversed.health -= 3
```

Более того, __`Traversable`__ композируемо с __`Lens`__:

```
(.) :: Lens'      a b -> Traversal' b c -> Traversal' a c
(.) :: Traversal' a b -> Lens'      b c -> Traversal' a c

units                  :: Lens'      Game [Unit]
units.traversed        :: Traversal' Game  Unit
units.traversed.health :: Traversal' Game  Int

ghci> :t units.traversed.health
units.traversed.health
  :: Applicative f => (Int -> f Int) -> Game -> f Game
```

По __`Traversable`__ нельзя применить __`Getter`__ элемента, так как элементов может быть много, поэтому нужно обозначить, как их свернуть:

```
type Getting r s a = (a -> Const r a) -> s -> Const r s

toListOf :: Getting (Endo [a]) s a -> s -> [a]
(^..)    :: s -> Getting (Endo [a]) s a -> [a] 

ghci> toListOf partyHP newState 
[7,12,5]

ghci> initialState^..partyHP
[10,15,8]
ghci> newState^..partyHP
[7,12,5]
```

__`filtered`__ сужает действия __`Traversal`__ по предикату:

```
around :: Point -> Double -> Traversal' Unit Unit
around center radius = filtered (\unit ->
    (unit^.position.x - center^.x)^2
  + (unit^.position.y - center^.y)^2
  < radius^2 )

fireBreath :: Point -> StateT Game IO ()
fireBreath target = do
    lift $ putStrLn "*rawr*"
    units.traversed.(around target 1.0).health -= 3

ghci> initialState^..units.traversed.position
[Point {_x = 3.5, _y = 7.0},Point {_x = 1.0, _y = 1.0},Point {_x
 = 0.0, _y = 2.1}]

ghci> newState <- execStateT (fireBreath (Point 0.5 1.5)) initialState 
*rawr*
ghci> (initialState^..partyHP, newState^..partyHP)
([10,15,8],[10,12,5])
```

__`zoom`__ позволяет сделать действия над объектом, не плодя лишних композиций линз:

```
partyLoc :: Traversal' Game Point
partyLoc = units.traversed.position

retreat :: StateT Game IO ()
retreat = do
    lift $ putStrLn "Retreat!"
    zoom partyLoc $ do
        x += 10
        y += 10
```

### Prisms

__`Prism`__ - это линза для работы с сумма-типами. 

```
type Prism s t a b 
    = forall p f. (Choice p, Applicative f) => p a (f b) -> p s (f t)

preview :: Prism' s a -> s -> Maybe a
review :: Prism' s a -> a -> s
```

Самый очевидный пример:

```
data X = MkX { _fieldX :: String }
data C = MkC1 X | MkC2 Y
data B = MkB { _fieldB :: C   }
data A = MkA { _fieldA :: [B] }

foo :: A

-- How to update fieldX inside A only if it's X? --
foo & fieldA.traversed.fieldB._MkC1.fieldX .~ "New string"
```

Ещё примеры:

```
_Left :: Prism' (Either a b) a
_Just :: Prism' (Maybe a) a

ghci> preview _Left (Left "hi")
Just "hi"
ghci> preview _Left (Right "hi")
Nothing
ghci> review _Left "hi"
Left "hi"
ghci> preview _Just (Just "hi")
Just "hi"
ghci> review _Just "hi"
Just "hi"
ghci> Left "hi" ^? _Left
Just "hi"

_Cons :: Prism' [a] (a, [a])
_Nil :: Prism' [a] ()

ghci> preview _Cons []
Nothing
ghci> preview _Cons [1,2,3]
Just (1, [2,3])
ghci> preview _Nil []
Just ()
ghci> preview _Nil [1,2,3]
Nothing
```

## Macros

Макросы - это инструменты генерации кода, встроенные в сам язык/компилятор. Как ни странно, GHC поддерживает препроцессор в полной мере:

```
{-# LANGUAGE CPP #-}

module Foo where

#define INT_SIZE 4

foo (INT_SIZE) = 2
foo _ = 4
```

Пример генерации кода:

```
{-# LANGUAGE CPP #-}

#define BASE(M)                           \
instance MonadBaseControl (M) (M) where { \
    type StM (M) a = a;                   \
    liftBaseWith f = f id;                \
    restoreM = return;                    \
    {-# INLINABLE liftBaseWith #-};       \
    {-# INLINABLE restoreM #-}}

BASE(IO)
BASE(Maybe)
BASE(Either e)
BASE([])
BASE((->) r)
BASE(Identity)
BASE(STM)

#if MIN_VERSION_base(4,4,0)
BASE(Strict.ST s)
BASE(       ST s)
#endif

#undef BASE
```

### Плюсы и минусы

__Плюсы__
- Полный контроль над тем, что в итоге получается 
- Максимальная мощность
- Получившееся будет гарантированно кодом на
заданном языке

__Минусы__
- Адски непонятный код реализации, который не похож на то, что в итоге генерируется

## Template Haskell

### Lens generator

В реальности линзы никто не пишет ручками, их генерирует компилятор. И вот как:

```
-- Подключим специальное расширение --
{-# LANGUAGE TemplateHaskell #-}

module MyModule where

-- Заимпортим библиотеку Control.Lens --
import Control.Lens

-- Озаглавим поля с '_' --
data Game = Game
     { _score :: Int
     , _units :: [Unit]
     , _boss  :: Unit
     } deriving (Show)

data Unit = Unit
     { _health   :: Int
     , _position :: Point
     } deriving (Show)

data Point = Point
     { _x :: Double
     , _y :: Double
     } deriving (Show)

-- Напишем соответствующие команды для генерации --
makeLenses ''Game
makeLenses ''Unit
makeLenses ''Point
```

Как итог, будет сгенерировано буквально вот это:

```
score :: Lens' Game Int
score = lens _score (\game v -> game { _score = v })

units :: Lens' Game [Unit]
units = lens _units (\game v -> game { _units = v })

boss :: Lens' Game Unit
boss = lens _boss (\game v -> game { _boss = v })

health :: Lens' Unit Int
health = lens _health (\unit v -> unit { _health = v })

position :: Lens' Unit Point
position = lens _position (\unit v -> unit { _position = v })

x :: Lens' Point Double
x = lens _x (\point v -> point { _x = v })

y :: Lens' Point Double
y = lens _y (\point v -> point { _y = v })
```

Удобно. Но это ещё не всё...

### Ещё немного генераций функций

Рассмотри генерацию функции __`fst`__ от кортежа ненулевой длины:

```
{-# LANGUAGE TemplateHaskell #-}
print $ $(fstN 3) ("hello world", 1, 2)
print $ $(fstN 4) ("hello world", 1, 2, 3)
```

А как это устроенно ?

```
{-# LANGUAGE TemplateHaskell #-}
module FstN where
import Language.Haskell.TH

fstN :: Int -> Q Exp
fstN n = do
   x <- newName "x"
   pure $ LamE [TupP $ VarP x : replicate (n - 1) WildP] (VarE x)
```

__What the hell is that ???__

Давайте разбираться. 

- __`VarP`__ - типа паттерн в аргументах функции.  
__`VarE`__ - типа хаскельное выражение.  
```
ghci> :t (VarP, VarE)
(VarP, VarE) :: (Name -> Pat, Name -> Exp)
```

- __`Name`__ - нужен для передачи в __`VarP`__ и __`VarE`__.  
__`Q`__ - монада, внутри которой это все должно присходить.  
```
ghci> :t newName
newName :: String -> Q Name
```

- Так выглядят монадические функции конструкторов выше.  
```
ghci> :t (varE, varP)
(varE, varP) :: (Name -> Q Exp, Name -> Q Pat)
```

- Тут мы объявили лямбду. Сказали, что у нее есть аргумент кортеж, перечислили паттерны элементов кортежа и сказали, что нужно вернуть. Мы выразили функцию как объект.  
```
LamE [TupP [VarP x_1,WildP,WildP]] (VarE x_1)  -- AST (Abstract Syntax Tree) of lambda \(x,_,_) -> x
```

- Тоже самое, но в виде монадических функций:  
```
import Language.Haskell.TH
fst3 = do
    x <- newName "x"
    lamE [tupP [varP x, wildP, wildP]] (varE x)
```

Пример работы:

```
ghci> :t fst3
fst3 :: Q Exp  -- Q is some monad

ghci> :t $(fst3)
$(fst3) :: (t2, t1, t) -> t2

ghci> $(fst3) ("hello", 10, True)
"hello"
```

### One Rule for Macros

Запрещено определять и сплайсить макрос в одном модуле по техническим причинам:

$$
∀ (M :: Macro) : module[M] ≠ module[$(M)]
$$
macros __`M`__ and its splice __`$(M)`__ must be defined in __different modules__.


### Квазиквотирование

__Квазиквотирование__ (Quasi quotes) помогает совершить обратное преобразовает в __AST__ (Abstract Syntax Tree).

```
ghci> :set -XQuasiQuotes 
ghci> import Language.Haskell.TH (runQ)
ghci> runQ [| \x -> x |]
LamE [ VarP x_0 ] ( VarE x_0 )
```

Синтаксис предельно простой: для обычных выражений пишешь __`[| … |]`__ и ловишь кайф.  
Для невыражений надо указать, что парсишь:

- Type quotes  
`[t| Int -> Int |] :: Q Type`
- Pattern quotes  
`[p| xs@(x:r) |] :: Q Pat`
- Declaration quotes  
`[d| data Pair a = Pair a a |] :: Q [Dec]`

Например, сгенерируем инстансы:

```
{-# LANGUAGE TemplateHaskell #-}

module CustomShow where

import Language.Haskell.TH

emptyShow :: Name -> Q [Dec]
emptyShow name = [d|instance Show $(conT name) where show _ = ""|]
```

В это время в другом модуле:

```
{-# LANGUAGE TemplateHaskell #-}

import CustomShow (emptyShow)

data MyData = MyData
     { foo :: String
     , bar :: Int
     }

emptyShow ''MyData  -- ''MyData == mkName "MyData"
-- ^ Expands to: instance Show MyData where show _ = ""

main = print $ MyData { foo = "bar", bar = 5 }
```